Turn chicken scratches into module.

Reference:
-http://www.cnccookbook.com/CCCNCGCodeG20G21MetricImperialUnitConversion.htm
- http://marlinfw.org/meta/gcode/
- https://github.com/grbl/grbl/wiki
- http://www.linuxcnc.org/docs/2.5/html/gcode/other-code.html

#### V0.061

In [14]:
class GCode(object):
    def __init__(self):
        self.linenumbering = False
        self.buffer = list()
        
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
        
    def save(self, filename):
        pass
    
    def send(self):
        pass
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    
    def __iter__(self):
        self.line = -1
        return self
    
    def __next__(self):
        self.line += 1
        if self.line >= len(self.buffer):
            raise StopIteration
        return self.buffer[self.line]

def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        self.buffer.append(cmd_str)  
    return cmd_fcn      

# Good core to start with.
commands=list()
# GCodes
for code in [0, 1, 2, 3, 4, 20, 21, 28, 90, 91]:
    commands.append("G{code}".format(code=code))
# MCodes
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))
# Commands to implement.
print(commands)

for command in commands:
    setattr(GCode, command, cmd_factory(command))

['G0', 'G1', 'G2', 'G3', 'G4', 'G20', 'G21', 'G28', 'G90', 'G91', 'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6']


In [15]:
program = GCode()
program.G21()
program.G91()

In [16]:
from IPython.display import display_html

In [17]:
display_html(program)

G21 
 G91

In [18]:
print(program)

G21 
G91 


In [19]:
for line in program:
    print(line)

G21 
G91 


In [65]:
class GCode(object):
    def __init__(self, machine = None, buffer=None):
        self.linenumbering = False
        self.machine = machine
        if buffer is None:
            self.buffer = list()
        else:
            self.buffer = buffer
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
        
    def save(self, filename):
        with open(filename, "w") as fid:
            print(str(self), file=fid)
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    
    def __add__(self, other):
        buffer = self.buffer
        buffer2 = other.buffer
        
        buffer.extend(buffer2)
        
        return GCode(machine=self.machine, buffer=buffer)
        
    def __iter__(self):
        self.line = -1
        return self
    
    
    def __next__(self):
        self.line += 1
        if self.line >= len(self.buffer):
            raise StopIteration
        return self.buffer[self.line]
    
    def optimise(self):
        """ Optimize the GCode """
        raise(NotImplemented(""))

def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        self.buffer.append(cmd_str)  
    return cmd_fcn      

# Good core to start with.
commands=list()
# GCodes
for code in [0, 1, 2, 3, 4, 20, 21, 28, 90, 91]:
    commands.append("G{code}".format(code=code))
# MCodes
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))
# Commands to implement.
print(commands)

for command in commands:
    setattr(GCode, command, cmd_factory(command))

['G0', 'G1', 'G2', 'G3', 'G4', 'G20', 'G21', 'G28', 'G90', 'G91', 'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6']


In [66]:
def square(size=20):
    program = GCode()
    program.G21()
    program.G91()
    program.G1(F=200)
    program.M4(S=25)
    program.G1(X=size)
    program.G1(Y=size)
    program.G1(X=-size)
    program.G1(X=-size)
    program.M5()
    return program

In [67]:
sqr20 = square(20)

In [68]:
sqr20

<GCode>[cmds=9]

In [69]:
sqr20.save("square_20mm.gcode")

In [61]:
# %load square_20mm.gcode
G21 
G91 
G1 F200
M4 S25
G1 X20
G1 Y20
G1 X-20
G1 X-20
M5 

SyntaxError: invalid syntax (<ipython-input-61-e1bf4b1a2558>, line 4)

In [70]:
sqr20 = square(20)
sqr40 = square(40)

In [72]:
sqr20+sqr40

<GCode>[cmds=18]